In [1]:
%run data_classes.ipynb


Imported classes.


/home/ptenkaate/.local/lib/python3.6/site-packages/matplotlib/backends/qt_editor/figureoptions.py:11: MatplotlibDeprecationWarning: 
The support for Qt4  was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  from matplotlib.backends.qt_compat import QtGui


In [8]:
from torch.utils.data import Dataset, DataLoader
import random

class PrepareData2D(object):
    def __init__(self, project_names, seed=34): 
        self.project_names = project_names
        self.seed = seed 
        
        self.project = self.load_project(self.project_names)
        
        #filter shape of the images, temporarily until further preprocessing
        self.project.filter_dimension_shape(0, 128, print_dropped=False)
        self.project.filter_dimension_shape(2, 24, print_dropped=False)
        
        self.project.normalize()
        
        self.proj_list = np.array(self.projects_list())
        self.subj_list = np.array(self.subjects_list())
        self.masks = np.dstack(self.project.masks)
        self.pcmras = np.dstack(self.project.pcmras)
        
        self.train, self.val, self.test = self.create_train_val_test_split()


    def load_project(self, project_names): 
        if type(project_names) == str: 
            project = Project(project_names)
        
        if type(project_names) == list: 
            if len(project_names) == 1: 
                project = Project(project_names[0])
            else:
                project = Project(project_names[0])
                for i in range(1, len(project_names)):
                    project.append_project(project_names[i])
        
        return project 
    
    
    def subjects_list(self):
        subj_list = []
        
        for i in range(len(self.project.subjects)):
            subject = self.project.subjects[i]
            for j in range(self.project.masks_shape[i][2]): 
                subj_list.append(subject)
        
        return subj_list

    
    def projects_list(self): 
        proj_list = []
        
        for i in range(len(self.project.subprojects)):
            subproject = self.project.subprojects[i]
            for j in range(self.project.masks_shape[i][2]): 
                proj_list.append(subproject)
        
        return proj_list
    
    import random 

    def create_train_val_test_split(self):

        random.seed(self.seed)

        idx = list(range(self.subj_list.shape[0]))
        random.shuffle(idx)

        split1 = int(len(idx) * 0.6)
        split2 = int(len(idx) * 0.8)

        train_idx = idx[:split1]
        val_idx = idx[split1:split2]
        test_idx = idx[split2:]

        train_data = (self.proj_list[train_idx], 
                      self.subj_list[train_idx], 
                      self.pcmras[:,:,train_idx], 
                      self.masks[:,:,train_idx])

        val_data = (self.proj_list[val_idx], 
                      self.subj_list[val_idx], 
                      self.pcmras[:,:,val_idx], 
                      self.masks[:,:,val_idx])

        test_data = (self.proj_list[test_idx], 
                      self.subj_list[test_idx], 
                      self.pcmras[:,:,test_idx], 
                      self.masks[:,:,test_idx])

        return train_data, val_data, test_data
    
    
    
class Dataset2D(Dataset):   
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data[0].shape[0]

    def __getitem__(self, idx):
        # returns: <project>, <subject>, <pcmra>, <mask>
        
        return (self.data[0][idx], 
                self.data[1][idx], 
                np.dstack(self.data[2][:, :, idx]), 
                np.dstack(self.data[3][:,:,idx]))
        
#         return (self.data[0][:, :, idx], 
#                 self.data[1][:,:,idx])

In [15]:
# data = PrepareData2D(["Aorta Volunteers", "Aorta BaV", "Aorta Resvcue", "Aorta CoA"])
# train_ds = Dataset2D(data.train)
# train_dl = DataLoader(train_ds, batch_size=32)

# for batch in train_dl: 
#     break

# print("Projects:", batch[0])
# print("\n Subjects:", batch[1])
# print("\n PCMRAs:", batch[2].shape)
# print("\n Masks:", batch[3].shape)

Masks without PCMRA: []
Masks without PCMRA: []
Masks without PCMRA: ['RESV_006.npy', 'RESV_011.npy', 'RESV_012.npy', 'RESV_014.npy', 'RESV_111.npy', 'RESV_204.npy', 'RESV_205.npy']
Masks without PCMRA: []
Projects: ('Aorta CoA', 'Aorta CoA', 'Aorta Volunteers', 'Aorta CoA', 'Aorta BaV', 'Aorta BaV', 'Aorta CoA', 'Aorta CoA', 'Aorta Volunteers', 'Aorta Resvcue', 'Aorta Resvcue', 'Aorta BaV', 'Aorta Resvcue', 'Aorta Resvcue', 'Aorta Resvcue', 'Aorta Volunteers', 'Aorta Resvcue', 'Aorta Resvcue', 'Aorta Volunteers', 'Aorta CoA', 'Aorta Resvcue', 'Aorta Volunteers', 'Aorta Volunteers', 'Aorta CoA', 'Aorta BaV', 'Aorta CoA', 'Aorta Resvcue', 'Aorta Resvcue', 'Aorta CoA', 'Aorta Resvcue', 'Aorta Volunteers', 'Aorta Volunteers')

 Subjects: ('16-06-15_224_done.npy', '16-05-18_215_done.npy', '16-01-22_Jarik_kt-pca_done2.npy', '16-07-06_216_done1.npy', '16-08-03_100_done1.npy', '16-04-20_105_done.npy', '16-04-20_212_done.npy', '16-09-09_241_done1.npy', '16-06-22_vanKesteren_kt_pca_done2.npy', 

In [22]:
def show_2d_batch(batch):
    title = [proj + ": " + subj for proj, subj in zip(batch[0], batch[1])]
    
    pcmra = batch[2].clone()
    pcmra = pcmra.reshape(pcmra.shape[0], pcmra.shape[2], pcmra.shape[3])
    pcmra = pcmra.permute(2, 1, 0).detach().numpy()
    
    mask = batch[3].clone()
    mask = mask.reshape(mask.shape[0], mask.shape[2], mask.shape[3])
    mask = mask.permute(2, 1, 0).detach().numpy()


    show = Show_images(title, (pcmra, "pcmra"), (mask, "mask"), (pcmra + mask, "pcmra + mask"))
    
    return show

In [23]:
# %matplotlib qt
# show = show_2d_batch(batch)